In [30]:
#!/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/bin/python


#Standard Header used on the projects
# %%

dataSize = 'big' #'big'  # 'small'

#first the major packages used for math and graphing
import numpy as np

import os as os
import random
import multiprocessing
from joblib import Parallel, delayed

import cv2
from sklearn.model_selection import train_test_split

import datetime

import platform

HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
    

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    if dataSize == 'big':
        folder = '/sciclone/scr10/dchendrickson01/CraneData/'
        imFolder ='/sciclone/scr10/dchendrickson01/BigData/'
    else:
        folder = '/sciclone/data10/dchendrickson01/SmallCopy/'
        imFolder = '/sciclone/data10/dchendrickson01/SmallCopy/'
elif Computer == "Desktop":
    rootfolder = location
    imFolder = "E:\\Backups\\Dan\\CraneData\\Images\\"
    if dataSize == 'big':
        folder = 'G:\\CraneData\\'
    else:
        folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'
    

In [2]:
scales = 500
#img_height , img_width = scales, 200
DoSomeFiles = False

SmoothType = 3  # 0 = none, 1 = rolling average, 2 = low pass filter, 3 = Kalman filter
WaveletToUse = 'beta'

num_cores = multiprocessing.cpu_count() -1
NumberOfFiles = num_cores - 2
GroupSize = NumberOfFiles


files = os.listdir(folder)
if DoSomeFiles: files = random.sample(files,NumberOfFiles*2)

In [ ]:
import CoreFunctions as cf

In [48]:
def resizeImage(FP):
    length = int(np.shape(FP)[0])
    width = int(np.shape(FP)[1]/6)
    res = cv2.resize(FP, dsize=(width, length), interpolation=cv2.INTER_LINEAR_EXACT)

    return res

def saveImage(FP, FName):
    cv2.imwrite(imFolder + FName + '.png', FP)
    return 1

In [4]:
def MakeImageFiles(files):
    numF = np.size(files)
    Keep = np.ones(numF)
    for i in range(numF):
        if os.path.isfile(imFolder + files[i] + '.png'):
            Keep[i]=0
    Keep = np.array(Keep, dtype=bool)   
    files=np.array(files)[Keep]
    
    AllAccels = Parallel(n_jobs=num_cores)(delayed(cf.getAcceleration)(file) for file in files)
    Flattened = []
    for j in range(np.shape(AllAccels)[0]):
        if AllAccels[j][0] == False:
            print(j,AllAccels[j][1])
        else: 
            Flattened.append(AllAccels[j])

    MetaData = []  #np.asarray([],dtype=object)
    DataOnlyMatrix = np.asarray([],dtype=object)
    for j in range(np.shape(AllAccels)[0]):
        if AllAccels[j][0] == False or np.shape(AllAccels[j][0][2])[0] != 60000:
            if AllAccels[j][1][4:9] =='Accel':
                print(j,AllAccels[j][1])
        else: 
            for k in range(3):
                MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1], AllAccels[j][k][3], AllAccels[j][k][4]])
                if np.size(DataOnlyMatrix) == 0:
                        DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
                else:
                        DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

    MetaData = np.matrix(MetaData)

    AllAccels = cf.KalmanGroup(DataOnlyMatrix)

    del DataOnlyMatrix

    maxes = np.amax(AllAccels[:,500:], axis = 1)
    mins = np.amin(AllAccels[:,500:], axis = 1)

    Keep = np.zeros(mins.size)
    for i in range(mins.size):
        if i % 3 == 0:
            if maxes[i] > 0.01 and mins[i] < -0.01:
                Keep[i]=1
                Keep[i+1]=1
                Keep[i+2]=1
                #print(i)


    Keep = np.array(Keep, dtype='bool')

    AllAccels = AllAccels[Keep,:]
    MetaData = MetaData[Keep,:]

    MotionsLeft = int(np.shape(AllAccels)[0]/3.0)

    AllFingers =  Parallel(n_jobs=num_cores)(delayed(cf.makeMPFast)([AllAccels[i*3],AllAccels[i*3+1],AllAccels[i*3+2]]) for i in range(MotionsLeft))
    del AllAccels

    SmallFingers =  Parallel(n_jobs=num_cores)(delayed(resizeImage)(FP) for FP in AllFingers)
    del AllFingers

    count =  Parallel(n_jobs=num_cores)(delayed(saveImage)(SmallFingers[i], MetaData[i*3,3]) for i in range(MotionsLeft))
    
    return sum(count)

In [5]:
GroupSize = NumberOfFiles

In [ ]:
fCount = len(files)
GroupsLeft = int(fCount/GroupSize) + 1

In [14]:
SplitRatio = 1/(GroupsLeft)

RemainingFiles, GroupFiles, x,y = train_test_split(files, range(len(files)), test_size=SplitRatio, shuffle=True, random_state=0)

In [15]:
GroupsLeft -=1

In [16]:
count = MakeImageFiles(GroupFiles)

starttime = datetime.datetime.now()
looptime = starttime
i = 1

/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

In [ ]:
while GroupsLeft > 1:
    SplitRatio = 1/(GroupsLeft)

    RemainingFiles, GroupFiles, x,y = train_test_split(RemainingFiles, range(len(RemainingFiles)), test_size=SplitRatio, shuffle=True, random_state=0)
       
    GroupsLeft -=1

    count = MakeImageFiles(GroupFiles)
    #saver.restore('model.ckpt')
    tNow = datetime.datetime.now()
    
    print(count,i,GroupsLeft, tNow-starttime, tNow-looptime)
        #saver.save(sess,'model.ckpt')
    i+=1
    looptime = tNow

/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

39 1 5318 0:04:03.146569 0:04:03.146569


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

33 2 5317 0:05:37.621523 0:01:34.474954


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

40 3 5316 0:07:24.397992 0:01:46.776469


/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


36 4 5315 0:09:01.247030 0:01:36.849038
35 5 5314 0:10:39.238262 0:01:37.991232
41 6 5313 0:12:29.069638 0:01:49.831376
29 7 5312 0:13:51.055759 0:01:21.986121
36 8 5311 0:15:28.679140 0:01:37.623381
31 9 5310 0:16:53.818063 0:01:25.138923
36 10 5309 0:18:33.072535 0:01:39.254472
35 11 5308 0:20:09.102013 0:01:36.029478
28 12 5307 0:21:30.523408 0:01:21.421395
31 13 5306 0:22:58.036959 0:01:27.513551
33 14 5305 0:24:29.679945 0:01:31.642986
30 15 5304 0:25:54.218438 0:01:24.538493
31 16 5303 0:27:19.680133 0:01:25.461695
33 17 5302 0:28:51.922649 0:01:32.242516
39 18 5301 0:30:36.598591 0:01:44.675942
38 19 5300 0:32:18.370655 0:01:41.772064
39 20 5299 0:34:03.038450 0:01:44.667795
37 21 5298 0:35:40.263099 0:01:37.224649


In [ ]:
count = MakeImageFiles(RemainingFiles)
tNow = datetime.datetime.now()
print(count,i,GroupsLeft, tNow-starttime, tNow-looptime)
print("Done")

In [18]:
SplitRatio = 1/(GroupsLeft)

RemainingFiles, GroupFiles, x,y = train_test_split(RemainingFiles, range(len(RemainingFiles)), test_size=SplitRatio, shuffle=True, random_state=0)

GroupsLeft -=1

files = GroupFiles

In [19]:
numF = np.size(files)
Keep = np.ones(numF)
for i in range(numF):
    if os.path.isfile(imFolder + files[i] + '.png'):
        Keep[i]=0
Keep = np.array(Keep, dtype=bool)   
files=np.array(files)[Keep]

In [20]:
AllAccels = Parallel(n_jobs=num_cores)(delayed(cf.getAcceleration)(file) for file in files)
Flattened = []
for j in range(np.shape(AllAccels)[0]):
    if AllAccels[j][0] == False:
        print(j,AllAccels[j][1])
    else: 
        Flattened.append(AllAccels[j])

In [21]:
MetaData = []  #np.asarray([],dtype=object)
DataOnlyMatrix = np.asarray([],dtype=object)
for j in range(np.shape(AllAccels)[0]):
    if AllAccels[j][0] == False or np.shape(AllAccels[j][0][2])[0] != 60000:
        if AllAccels[j][1][4:9] =='Accel':
            print(j,AllAccels[j][1])
    else: 
        for k in range(3):
            MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1], AllAccels[j][k][3], AllAccels[j][k][4]])
            if np.size(DataOnlyMatrix) == 0:
                    DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
            else:
                    DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

In [22]:
MetaData = np.matrix(MetaData)

AllAccels = cf.KalmanGroup(DataOnlyMatrix)

del DataOnlyMatrix

maxes = np.amax(AllAccels[:,500:], axis = 1)
mins = np.amin(AllAccels[:,500:], axis = 1)

Keep = np.zeros(mins.size)
for i in range(mins.size):
    if i % 3 == 0:
        if maxes[i] > 0.01 and mins[i] < -0.01:
            Keep[i]=1
            Keep[i+1]=1
            Keep[i+2]=1
            #print(i)

/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could

In [24]:
Keep = np.array(Keep, dtype='bool')

AllAccels = AllAccels[Keep,:]
MetaData = MetaData[Keep,:]

print(Keep)

[ True  True  True  True  True  True  True  True  True False False False
 False False False  True  True  True  True  True  True False False False
  True  True  True False False False  True  True  True False False False
 False False False False False False False False False False False False
 False False False  True  True  True  True  True  True False False False
  True  True  True  True  True  True False False False False False False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False False False  True  True  True False False False
 False False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False False False False False False False False False  True  True  True
  True  True  True  True  True  True False False False  True  True  True
 False False False False False False False False False False False False
  True  True  True False False False False False Fa

In [34]:
MotionsLeft = int(np.shape(AllAccels)[0]/3.0)

AllFingers =  Parallel(n_jobs=num_cores)(delayed(cf.makeMatrixImages)([AllAccels[i*3],AllAccels[i*3+1],AllAccels[i*3+2]]) for i in range(MotionsLeft))

In [49]:
SmallFingers =  Parallel(n_jobs=num_cores)(delayed(resizeImage)(FP) for FP in AllFingers)

In [45]:
np.shape(AllFingers)

(31, 500, 60000, 3)

In [43]:
count =  Parallel(n_jobs=num_cores)(delayed(saveImage)(SmallFingers[i], MetaData[i*3,3]) for i in range(MotionsLeft))

In [50]:
np.shape(SmallFingers)

(31, 500, 10000, 3)

In [51]:
count =  Parallel(n_jobs=num_cores)(delayed(saveImage)(SmallFingers[i], MetaData[i*3,3]) for i in range(MotionsLeft))